In [110]:
import numpy as np
import pandas as pd
import random
import copy
import math

In [111]:
# import the dataset
iris = np.genfromtxt('iris.data', delimiter=',', dtype = 'str')
iris = iris.tolist()

# Check if the import is valid
#iris

In [112]:
# fix the random seed according to the instruction
random.seed(17)

# Call random shuffle
random.shuffle(iris)

# Check the result
#iris

In [113]:
# cross validation
total_number = len(iris)
##total_number # Check
cross = []
for i in range(1,6):
    cross.append(iris[30*(i-1):30*i]) # Split the dataset

# Check the result
#cross
#print(type(cross[0]))
#cross[0][0][:4]
#cross[0][:,0]
#cross[:][:][:2]
##len(cross)
##len(cross[0])

In [116]:
# Initialization
# Create the augmented input feature vector
aug_cross = copy.deepcopy(cross)
for i in range(len(cross)):
  for j in range(len(cross[i])):
    #print(cross[i][j][:4]) # Code for debugging
    new_item = [eval(l) for l in cross[i][j][:4]]
    #new_item = cross[i][j][:4] # Code for debugging
    new_item.append(1)
    aug_cross[i][j]=new_item

# Check the result
#aug_cross
#cross
#aug_cross[0][0]

In [117]:
# Create vector for one-hot labels
#one_hot_label = cross.copy()
one_hot_label = copy.deepcopy(cross)
for i in range(len(cross)):
  for j in range(len(cross[i])):
    #print(cross[i][j][4]) # Code for debug
    if cross[i][j][4] == 'Iris-setosa':
      one_hot_label[i][j]=[1,0,0]
    elif cross[i][j][4] == 'Iris-versicolor':
      one_hot_label[i][j]=[0,1,0]
    else:
      one_hot_label[i][j]=[0,0,1]

# Check the result
#one_hot_label[0][0]

In [118]:
# Initialize Weight of MLP (Random Initialization with Uniform Distribution)
D = 5 # Number of the Input Nodes (Features)
H = 20 # Number of the Hidden Nodes
L = 3 # number of the Output Nodes (Classes)
# Calculate the upper and lower bound of Uniform Distribution
Up1 = math.sqrt(6/(D+1+H))
Up2 = math.sqrt(6/(H+1+L))
Low1 = -Up1
Low2 = -Up2
# Initialize the weight
weights1 = np.random.uniform(low=Low1, high=Up1, size=(D, H)).tolist()
weights2 = np.random.uniform(low=Low2, high=Up2, size=(H, L)).tolist()

# Check the result
#print(weights1[0])
#print(len(weights2))
#print(weights2)

In [119]:
# Define a function for Forward Propagation
def forward_propagation(x, weights, one_hot_label):
  a = []
  z = []
  y = []
  E = 0 # Initial Value for the Loss Function
  new_item = 0

  # Calculate a
  for j in range(20):
    for i in range(5):
      # Code for debug
      #print(weights1[i][j])
      new_item += weights1[i][j]*x[i]
    a.append(new_item)

  # Calculate z
  for j in range(20):
    new_item = math.tanh(a[j])
    z.append(new_item)

  # Calculate y
  for k in range(3):
    for j in range(20):
      new_item += weights2[j][k]*z[j]
    y.append(new_item)

  # Calculate the loss function
  for k in range(3):
    #print(y[k])
    #print(type(y[k]))
    #print(type(one_hot_label[k]))
    E += 0.5*(y[k]-one_hot_label[k])**2

  # Debugger
  #return a
  #return z
  #return y
  #return E

  return a,z,y,E

# Check the result
#forward_propagation(aug_cross[0][0], weights1[0], one_hot_label[0][0])

In [120]:
# Define a function for Backward Propagation
def backward_propagation(x,a,z,y,E, one_hot_label):
  # Evaluate delta_k
  # Initialize value for delta_k
  delta_k = []
  new_item = 0
  for k in range(3):
    new_item = y[k]-one_hot_label[k]
    delta_k.append(new_item)

  # Initialize delta_j
  delta_j = []
  for j in range(20):
    for k in range(3):
      new_item += (1-z[j]**2) * weights2[j][k]*delta_k[k]
    delta_j.append(new_item)

  # Initialize partial derivative 1 and 2
  pd1 = np.zeros((4,20)).tolist()
  pd2 = np.zeros((20,3)).tolist()
  #print(type(partial_derivative_1))

  # Calculate pd1
  for i in range(4):
    for j in range(20):
      pd1[i][j] = delta_j[j]*x[i]

  # Calculate pd2
  for j in range(20):
    for k in range(3):
      pd2[j][k] = delta_k[k]*z[j]


  # Check the result
  #return delta_k
  #return delta_j
  #return pd1
  #return pd2
  return delta_k,delta_j,pd1,pd2

# Check the result
#a_z_y_E = list(forward_propagation(aug_cross[0][0], weights1[0], one_hot_label[0][0]))
#print(a_z_y_E)
#backward_propagation((aug_cross[0][0]),a_z_y_E[0],a_z_y_E[1],a_z_y_E[2],a_z_y_E[3],one_hot_label[0][0])

In [ ]:
# Implement the Stochastic Backpropagation Algorithm

# Omega has already been initialized above

# Initialize eta
eta = 0.001

# Initialize train and test set
train_set = []
test_set = []

# Obtain the train and test set
for t in range(5):
  test_set = cross[t]
  for tt in range(5):
    if tt == t:
      pass
    else:
      train_set += cross[tt]

  # Random Shuffle the train set
  random.shuffle(train_set)